# Data Input and Homogenisation

## 1. Types of input data for text corpora

Textual data might come in different forms. 

1. It could be **plain text**:

```
Die Grippe wütet weiter
Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten beiden Tagen auch in Groß-Berlin noch deutlich gestiegen. Die Warenhäuser und sonstigen Geschäfte, die Kriegs- und die privaten Betriebe klagen, dass übermäßig viele Angestellte krank melden müssen, und auch bei der Post und bei der Straßenbahn ist die Zahl der Grippekranken bedeutend gestiegen.
```

2. It could be **images** (pdf, jpg, etc):

<img src="images/grippe1.png" width=700>

(source: Berliner Morgenpost, October 15, 1918)

3. It could be some **structured markup** (XML/HTML):

```
<text>
    <head>
        Die Grippe wütet weiter
    </head>
    <p>
        <s>Zunahme der schweren Fälle in Berlin.</s> 
        <s>Die Zahl der Grippefälle ist in den letzten beiden Tagen auch in Groß-Berlin noch deutlich gestiegen.</s>
        <s>Die Warenhäuser und sonstigen Geschäfte, die Kriegs- und die privaten Betriebe klagen, dass übermäßig viele Angestellte krank melden müssen, und auch bei der Post und bei der Straßenbahn ist die Zahl der Grippekranken bedeutend gestiegen.</s>
    </p>
</text>
```

#### We have to be able to use all these formats and homogenise different sources into a unified corpus 

## 2.  images into digial text. OCR

To process images into digital text we need an **Optical Character Recognition (OCR)** tool. There are some commercial ones (like FineReader), we'll use an open & free tool in Python. 

In [1]:
#!pip install pytesseract
#!pip install pillow

In [2]:
import pytesseract

In [3]:
from PIL import Image

### 2.1. Evaluate OCR engine quality

Before processing all files, we evaluate OCR quality on a sample image part for evaluation(source: [Deutsche Zeitung, Ausgaben am Montag, 23.12.1918](https://zefys.staatsbibliothek-berlin.de/kalender/auswahl/date/1918-12-23/30744015/)):
![sample.jpg](sample.jpg)

Let us OCR it:

In [4]:
ocr_output = pytesseract.image_to_string(Image.open('images/sample.jpg'), lang='frk')  # using German fraktur OCR model

In [5]:
print(ocr_output)

Die Lage anfdemKohlenmarkte gibt zu “en ſhlimm-
ſten Befürc<tungen Anlaß. Für Sachſen fehlten im Nov»mber
30 000 Wagen zu je 10 Tonnen und für Tezembex wird mit no<
größeren Ausfällen gere<net werden. E3 iſt mit einem völligen
Stillſtand der Induſtrie innerhalb vierzehn Tagen zu red<hnen,
wenn nicht eine erhebliche Steigerung der Belenſ<aften der Kot:en-
bergwerke oder ihrer Zah! geiingt. Weiter ſteht eine weſentliche
Erhöhung der Kohlenpreije bevor.



#### 2.1.1 Manually create  the 'ground truth' to evaluate against

In [6]:
ground_truth = input('Please insert corrected string: ')

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
print(ground_truth)

#### 2.1.2 Measure OCR precision, recall and F-measure

In [ ]:
import Levenshtein as lev

In [ ]:
def measure_quality(ocr_output, ground_truth):
    """
    Calculates precision, recall, and F1-score
    using the Levenshtein distance to align text from OCR with the ground truth data.

    :param ocr_output: A string containing the raw OCR results.
    :param ground_truth: A string containing the verified ground truth text.
    """

    matching_parts = lev.matching_blocks(lev.editops(ocr_output, ground_truth), ocr_output, ground_truth)
    true_pos = len(''.join([ocr_output[x[0]:x[0]+x[2]] for x in matching_parts]))

    precision = true_pos / len(ground_truth)
    recall = true_pos / len(ocr_output)
    f_score = 2 * ((precision * recall) / (precision + recall))

    return precision, recall, f_score

In [ ]:
precision, recall, f_score = measure_quality(ocr_output, ground_truth)

In [ ]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

## Sidenote: GPT4 does a pretty good job of OCR post-correction:

### Image:

<img src="images/grippe1.png" width=500>

### Raw OCR-ed text

In [ ]:
ocr_output = pytesseract.image_to_string(Image.open('images/grippe1.png'), lang='frk') 

In [ ]:
print(ocr_output)

### post-correction with GPT4:

<img src="images/gpt_postcorr.png" width=700>

### Сleaner text:

Grippe wütet weiter

Zunahme der schweren Fälle in Berlin.

Die Zahl der Grippefälle ist in den letzten beiden Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, dass übermäßig viele Angestellte krank melden müssen, und auch bei der Post und bei der Straßenbahn ist die Zahl der Grippekranken bedeutend gewachsen.

#### Perhaps we shouldn't encourage students to do this at this point... But it's good to be aware of this.
The way we do OCR might change quickly in the coming years

### 2.2 Process the whole corpus of PDF-s with the same OCR engine

In [ ]:
import os
from tqdm import tqdm
from pdf2image import convert_from_path

In [ ]:
pathpdf = '../data/pdf'

In [ ]:
for filename in tqdm(os.listdir(pathpdf)):
    if '.pdf' in filename:
        thispath = os.path.join(pathpdf, filename)
        converted_pdf = convert_from_path(thispath, use_cropbox=True)
        with open(thispath.replace('.pdf', '.txt'), 'w') as output_txt:
            for image in converted_pdf:
                recognized = pytesseract.image_to_string(image, 
                                                         lang='frk') 
                output_txt.write(recognized)

#### After running this we have all our PDF-s in plain txt form

## 3.  Getting digial text from the structured markup (XML)

Unlike text on the image, XML/HTML are already machine readable, so they are a lower-hanging fruit. Still, we'll need to use a parser for such markup to get rid of XML/HTML tags and some metadata

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
pathtoxmlfiles = '../data/xml'

In [ ]:
for filename in os.listdir(pathtoxmlfiles):
    if '.xml' in filename:
        path2file = os.path.join(pathtoxmlfiles, filename)
        with open(path2file) as openxml:
            soup = BeautifulSoup(openxml)
        print(soup.find('text').text.strip())
        #with open(path2file.replace('.txt', '.xml'), 'w') as output_xml:
        #    output.write(soup.find('text').text.strip())
            

#### After running this we have all our XML-s in plain txt form

## Now let's use all the data for processing and analysis (next notebook)